## libraries

In [14]:
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
!cp -r /kaggle/input/ir-dataset/* /kaggle/working/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


##  task 1

In [20]:
! gdown 'https://drive.google.com/uc?id=1cbXtZBzBMRmLsBX4W08pCWnK0dJNYTun'
! tar -xvzf /kaggle/working/citation_mapping.tar.gz
!gdown 'https://drive.google.com/uc?id=1DHw-DdRB8xjTqvK-jU3Sol_-8VG4ACZK'
!tar -xvzf /kaggle/working/embeddings.tar.gz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading...
From: https://drive.google.com/uc?id=1cbXtZBzBMRmLsBX4W08pCWnK0dJNYTun
To: /kaggle/working/citation_mapping.tar.gz
100%|█████████████████████████████████████████| 264k/264k [00:00<00:00, 107MB/s]
Citation_JSONs/
Citation_JSONs/Citation_Train.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading...
From (original): https://drive.google.com/uc?id=1DHw-DdRB8xjTqvK-jU3Sol_-8VG4ACZK
From (redirected): https://drive.google.com/uc?id=1DHw-DdRB8xjTqvK-jU3Sol_-8VG4ACZK&confirm=t&uuid=508432eb-8c35-451f-a466-0c2e8c234a9f
To: /kaggle/working/embeddings.tar.gz
100%|████████████████████████████████████████| 316M/316M [00:04<00:00, 73.0MB/s]
embeddings_precalculated_docs/
embeddings_precalculated_docs/embeddings_PatentSBERTa_mean_TA.npy


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


embeddings_precalculated_docs/app_ids_PatentSBERTa_mean_TA.json
embeddings_precalculated_docs/embeddings_PatentSBERTa_mean_claims.npy
embeddings_precalculated_docs/app_ids_PatentSBERTa_mean_claims.json
embeddings_precalculated_docs/embeddings_PatentSBERTa_mean_TAC.npy
embeddings_precalculated_docs/app_ids_PatentSBERTa_mean_TAC.json
embeddings_precalculated_docs/embeddings_all-MiniLM-L6-v2_mean_TA.npy
embeddings_precalculated_docs/app_ids_all-MiniLM-L6-v2_mean_TA.json
embeddings_precalculated_docs/embeddings_all-MiniLM-L6-v2_mean_claims.npy
embeddings_precalculated_docs/app_ids_all-MiniLM-L6-v2_mean_claims.json
embeddings_precalculated_docs/embeddings_all-MiniLM-L6-v2_mean_TAC.npy
embeddings_precalculated_docs/app_ids_all-MiniLM-L6-v2_mean_TAC.json
embeddings_precalculated_train/
embeddings_precalculated_train/embeddings_PatentSBERTa_mean_TA.npy
embeddings_precalculated_train/app_ids_PatentSBERTa_mean_TA.json
embeddings_precalculated_train/embeddings_PatentSBERTa_mean_claims.npy
embeddi

In [21]:
import os
import json
import numpy as np
import torch
import faiss
from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split

# ----------------------------
# Model Settings and Parameters
# ----------------------------
MODEL_NAME = "PatentSBERTa"  # changed from "all-MiniLM-L6-v2" to "PatentSBERTa"
CONTENT_TYPE = "TA"          # e.g., "TA", "claims", or "TAC"
POOLING = "mean"             # pooling strategy
# Set QUERY_SET to either "train", "test", or "split"
QUERY_SET = "test"  
SAVE_RESULTS = True
TOP_N = 100
K_VALUE = 10  # for evaluation (used in some functions)
METRICS_TYPE = "all"
SPLIT_RATIO = 0.8  # For "split" mode: fraction for training

# ----------------------------
# Paths and Files
# ----------------------------
BASE_DIR = "/kaggle/working/"
DOC_EMBEDDING_DIR = os.path.join(BASE_DIR, "embeddings_precalculated_docs")
TRAIN_EMBEDDING_DIR = os.path.join(BASE_DIR, "embeddings_precalculated_train")
TEST_EMBEDDING_DIR = os.path.join(BASE_DIR, "embeddings_precalculated_test")
OUTPUT_DIR = os.path.join(BASE_DIR, "results")
CITATION_FILE = os.path.join(BASE_DIR, "Citation_JSONs/Citation_Train.json")

# Primary content type ("TA")
CONTENT_TYPE_1 = "TA"
DOC_EMBEDDING_FILE_1 = os.path.join(DOC_EMBEDDING_DIR, f"embeddings_{MODEL_NAME}_{POOLING}_{CONTENT_TYPE_1}.npy")
DOC_APP_IDS_FILE_1 = os.path.join(DOC_EMBEDDING_DIR, f"app_ids_{MODEL_NAME}_{POOLING}_{CONTENT_TYPE_1}.json")
QUERY_EMBEDDING_DIR = TRAIN_EMBEDDING_DIR if QUERY_SET != "test" else TEST_EMBEDDING_DIR
QUERY_EMBEDDING_FILE_1 = os.path.join(QUERY_EMBEDDING_DIR, f"embeddings_{MODEL_NAME}_{POOLING}_{CONTENT_TYPE_1}.npy")
QUERY_APP_IDS_FILE_1 = os.path.join(QUERY_EMBEDDING_DIR, f"app_ids_{MODEL_NAME}_{POOLING}_{CONTENT_TYPE_1}.json")

# Secondary content type (e.g., "claims")
CONTENT_TYPE_2 = "claims"
DOC_EMBEDDING_FILE_2 = os.path.join(DOC_EMBEDDING_DIR, f"embeddings_{MODEL_NAME}_{POOLING}_{CONTENT_TYPE_2}.npy")
DOC_APP_IDS_FILE_2 = os.path.join(DOC_EMBEDDING_DIR, f"app_ids_{MODEL_NAME}_{POOLING}_{CONTENT_TYPE_2}.json")
QUERY_EMBEDDING_FILE_2 = os.path.join(QUERY_EMBEDDING_DIR, f"embeddings_{MODEL_NAME}_{POOLING}_{CONTENT_TYPE_2}.npy")
QUERY_APP_IDS_FILE_2 = os.path.join(QUERY_EMBEDDING_DIR, f"app_ids_{MODEL_NAME}_{POOLING}_{CONTENT_TYPE_2}.json")

# ----------------------------
# Device Setup
# ----------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ----------------------------
# Utility Functions
# ----------------------------
def load_embeddings_and_ids(embedding_file, app_ids_file):
    print(f"Loading embeddings from {embedding_file}")
    embeddings = torch.from_numpy(np.load(embedding_file))
    with open(app_ids_file, 'r') as f:
        app_ids = json.load(f)
    print(f"Loaded {len(embeddings)} embeddings and {len(app_ids)} app_ids")
    return embeddings, app_ids

def citation_to_citing_to_cited_dict(citations):
    citing_to_cited_dict = {}
    for citation in citations:
        citing_to_cited_dict.setdefault(citation[0], []).append(citation[2])
    return citing_to_cited_dict

def get_true_and_predicted(citing_to_cited_dict, recommendations_dict):
    true_labels = []
    predicted_labels = []
    not_in_citation_mapping = 0
    for citing_id in recommendations_dict.keys():
        if citing_id in citing_to_cited_dict:
            true_labels.append(citing_to_cited_dict[citing_id])
            predicted_labels.append(recommendations_dict[citing_id])
        else:
            print(citing_id, "not in citation mapping")
            not_in_citation_mapping += 1
    return true_labels, predicted_labels, not_in_citation_mapping

# ----------------------------
# Metric Functions
# ----------------------------
def recall_at_k(true, pred, k):
    true_set = set(true)
    if len(true_set) == 0:
        return 0
    return len([d for d in pred[:k] if d in true_set]) / len(true_set)

def precision_at_k(true, pred, k):
    true_set = set(true)
    return len([d for d in pred[:k] if d in true_set]) / k

def average_precision(true, pred):
    true_set = set(true)
    if len(true_set) == 0:
        return 0
    score = 0.0
    num_hits = 0.0
    for i, doc in enumerate(pred):
        if doc in true_set:
            num_hits += 1
            score += num_hits / (i + 1)
    return score / len(true_set)

def compute_map(true_labels, predicted_labels):
    aps = [average_precision(t, p) for t, p in zip(true_labels, predicted_labels)]
    return np.mean(aps) if aps else 0

def balanced_accuracy(true, pred, total_docs):
    true_set = set(true)
    predicted_set = set(pred)
    TP = len(true_set & predicted_set)
    FP = len(predicted_set) - TP
    FN = len(true_set) - TP
    TN = total_docs - len(true_set) - FP
    if (TP + FN) == 0 or (TN + FP) == 0:
        return 0
    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    return (sensitivity + specificity) / 2

# ----------------------------
# Build FAISS Index Functions
# ----------------------------
def build_faiss_index(embeddings_np):
    # Ensure embeddings are float32 and normalized for cosine similarity via inner product.
    embeddings_np = embeddings_np.astype('float32')
    faiss.normalize_L2(embeddings_np)
    index = faiss.IndexFlatIP(embeddings_np.shape[1])
    index.add(embeddings_np)
    return index

def faiss_search(index, query_embeddings_np, top_n):
    query_embeddings_np = query_embeddings_np.astype('float32')
    faiss.normalize_L2(query_embeddings_np)
    distances, indices = index.search(query_embeddings_np, top_n)
    return distances, indices

# ----------------------------
# Load Embeddings and IDs
# ----------------------------
doc_embeddings_1, doc_app_ids_1 = load_embeddings_and_ids(DOC_EMBEDDING_FILE_1, DOC_APP_IDS_FILE_1)
query_embeddings_1, query_app_ids_1 = load_embeddings_and_ids(QUERY_EMBEDDING_FILE_1, QUERY_APP_IDS_FILE_1)
doc_embeddings_1 = doc_embeddings_1.to(device)
query_embeddings_1 = query_embeddings_1.to(device)

if os.path.exists(DOC_EMBEDDING_FILE_2) and os.path.exists(QUERY_EMBEDDING_FILE_2):
    doc_embeddings_2, doc_app_ids_2 = load_embeddings_and_ids(DOC_EMBEDDING_FILE_2, DOC_APP_IDS_FILE_2)
    query_embeddings_2, query_app_ids_2 = load_embeddings_and_ids(QUERY_EMBEDDING_FILE_2, QUERY_APP_IDS_FILE_2)
    doc_embeddings_2 = doc_embeddings_2.to(device)
    query_embeddings_2 = query_embeddings_2.to(device)
else:
    print("Secondary embeddings not found; using primary only.")
    doc_embeddings_2, doc_app_ids_2 = doc_embeddings_1, doc_app_ids_1
    query_embeddings_2, query_app_ids_2 = query_embeddings_1, query_app_ids_1

assert query_app_ids_1 == query_app_ids_2, "Mismatch between primary and secondary query app_ids!"

# ----------------------------
# Align Document Embeddings
# ----------------------------
# Only use documents that are common between both embedding types.
common_doc_ids = list(set(doc_app_ids_1) & set(doc_app_ids_2))
indices_1 = [doc_app_ids_1.index(doc_id) for doc_id in common_doc_ids]
indices_2 = [doc_app_ids_2.index(doc_id) for doc_id in common_doc_ids]
doc_embeddings_1_aligned = doc_embeddings_1[indices_1]
doc_embeddings_2_aligned = doc_embeddings_2[indices_2]

# ----------------------------
# Compute Rankings using FAISS (Task 1: TA method)
# ----------------------------
# Convert tensors to numpy arrays (on CPU) and ensure type float32.
doc_emb_1_np = doc_embeddings_1_aligned.cpu().numpy()
query_emb_1_np = query_embeddings_1.cpu().numpy()

# Build FAISS index and search.
index_TA = build_faiss_index(doc_emb_1_np)
_, I_TA = faiss_search(index_TA, query_emb_1_np, TOP_N)
results_TA = {}
for i, q_id in enumerate(query_app_ids_1):
    results_TA[q_id] = [common_doc_ids[idx] for idx in I_TA[i]]

# ----------------------------
# Compare Query IDs with Prediction Keys
# ----------------------------
test_query_ids = set(query_app_ids_1)
prediction_keys = set(results_TA.keys())
if test_query_ids != prediction_keys:
    missing_in_predictions = test_query_ids - prediction_keys
    extra_in_predictions = prediction_keys - test_query_ids
    print("Mismatch found!")
    if missing_in_predictions:
        print("The following query IDs are missing in predictions:", missing_in_predictions)
    if extra_in_predictions:
        print("The following keys are in predictions but not in the test query IDs:", extra_in_predictions)
else:
    print("All test query IDs are present in the predictions!")

# ----------------------------
# Evaluation on Training and Testing (Validation) Splits for Task 1
# ----------------------------
if QUERY_SET == "split":
    # Split queries into training and testing sets
    num_queries = len(query_app_ids_1)
    indices = np.arange(num_queries)
    np.random.shuffle(indices)
    split_point = int(num_queries * SPLIT_RATIO)
    train_indices = indices[:split_point]
    test_indices = indices[split_point:]
    
    train_query_emb1 = query_embeddings_1[train_indices]
    train_query_ids = [query_app_ids_1[i] for i in train_indices]
    test_query_emb1 = query_embeddings_1[test_indices]
    test_query_ids = [query_app_ids_1[i] for i in test_indices]
    
    # Use FAISS search for training and testing splits (using TA method)
    train_query_emb1_np = train_query_emb1.cpu().numpy()
    _, I_train = faiss_search(index_TA, train_query_emb1_np, TOP_N)
    results_train = {}
    for i, q_id in enumerate(train_query_ids):
        results_train[q_id] = [common_doc_ids[idx] for idx in I_train[i]]
    
    test_query_emb1_np = test_query_emb1.cpu().numpy()
    _, I_test = faiss_search(index_TA, test_query_emb1_np, TOP_N)
    results_test = {}
    for i, q_id in enumerate(test_query_ids):
        results_test[q_id] = [common_doc_ids[idx] for idx in I_test[i]]
    
    # Load citation mapping for evaluation
    with open(CITATION_FILE, 'r') as f:
        citations = json.load(f)
    citing_to_cited_dict = citation_to_citing_to_cited_dict(citations)
    
    total_docs = len(common_doc_ids)
    
    # --- Task 1 Evaluation ---
    # For Task 1, compute Recall@10, @20, @50, and @100; MAP; balanced accuracy and precision (top 100)
    # Training split
    true_labels_train, predicted_labels_train, _ = get_true_and_predicted(citing_to_cited_dict, results_train)
    recall_train_10 = np.mean([recall_at_k(t, p, 10) for t, p in zip(true_labels_train, predicted_labels_train)])
    recall_train_20 = np.mean([recall_at_k(t, p, 20) for t, p in zip(true_labels_train, predicted_labels_train)])
    recall_train_50 = np.mean([recall_at_k(t, p, 50) for t, p in zip(true_labels_train, predicted_labels_train)])
    recall_train_100 = np.mean([recall_at_k(t, p, 100) for t, p in zip(true_labels_train, predicted_labels_train)])
    map_train = compute_map(true_labels_train, predicted_labels_train)
    bal_acc_train = np.mean([balanced_accuracy(t, p[:100], total_docs) for t, p in zip(true_labels_train, predicted_labels_train)])
    prec_train = np.mean([precision_at_k(t, p, 100) for t, p in zip(true_labels_train, predicted_labels_train)])
    print("\nTask 1 Metrics (Training Split):")
    print(f"Recall@10: {recall_train_10:.4f}")
    print(f"Recall@20: {recall_train_20:.4f}")
    print(f"Recall@50: {recall_train_50:.4f}")
    print(f"Recall@100: {recall_train_100:.4f}")
    print(f"Mean Average Precision: {map_train:.4f}")
    print(f"Balanced Accuracy (top 100): {bal_acc_train:.4f}")
    print(f"Precision (top 100): {prec_train:.4f}")
    
    # Testing split
    true_labels_test, predicted_labels_test, _ = get_true_and_predicted(citing_to_cited_dict, results_test)
    recall_test_10 = np.mean([recall_at_k(t, p, 10) for t, p in zip(true_labels_test, predicted_labels_test)])
    recall_test_20 = np.mean([recall_at_k(t, p, 20) for t, p in zip(true_labels_test, predicted_labels_test)])
    recall_test_50 = np.mean([recall_at_k(t, p, 50) for t, p in zip(true_labels_test, predicted_labels_test)])
    recall_test_100 = np.mean([recall_at_k(t, p, 100) for t, p in zip(true_labels_test, predicted_labels_test)])
    map_test = compute_map(true_labels_test, predicted_labels_test)
    bal_acc_test = np.mean([balanced_accuracy(t, p[:100], total_docs) for t, p in zip(true_labels_test, predicted_labels_test)])
    prec_test = np.mean([precision_at_k(t, p, 100) for t, p in zip(true_labels_test, predicted_labels_test)])
    print("\nTask 1 Metrics (Testing Split):")
    print(f"Recall@10: {recall_test_10:.4f}")
    print(f"Recall@20: {recall_test_20:.4f}")
    print(f"Recall@50: {recall_test_50:.4f}")
    print(f"Recall@100: {recall_test_100:.4f}")
    print(f"Mean Average Precision: {map_test:.4f}")
    print(f"Balanced Accuracy (top 100): {bal_acc_test:.4f}")
    print(f"Precision (top 100): {prec_test:.4f}")

# ----------------------------
# Save Outputs (Task 1 Only)
# ----------------------------
if SAVE_RESULTS:
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    task1_output_file = os.path.join(OUTPUT_DIR, "prediction1.json")
    with open(task1_output_file, 'w') as f:
        json.dump(results_TA, f)
    print(f"\nSaved Task 1 output to {task1_output_file}")


Loading embeddings from /kaggle/working/embeddings_precalculated_docs/embeddings_PatentSBERTa_mean_TA.npy
Loaded 16837 embeddings and 16837 app_ids
Loading embeddings from /kaggle/working/embeddings_precalculated_test/embeddings_PatentSBERTa_mean_TA.npy
Loaded 1000 embeddings and 1000 app_ids
Loading embeddings from /kaggle/working/embeddings_precalculated_docs/embeddings_PatentSBERTa_mean_claims.npy
Loaded 16834 embeddings and 16834 app_ids
Loading embeddings from /kaggle/working/embeddings_precalculated_test/embeddings_PatentSBERTa_mean_claims.npy
Loaded 1000 embeddings and 1000 app_ids
All test query IDs are present in the predictions!

Saved Task 1 output to /kaggle/working/results/prediction1.json


## task 2 

In [22]:
import os
import json
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, models, util
from tqdm import tqdm
# Removed: import argparse
from pathlib import Path
import types # Used for SimpleNamespace if preferred

config = {
    # --- Data Files ---
    'base_dir': '.', # Base directory containing the data files.
    'query_list_file': 'test_queries.json', # Path to the JSON file with query IDs (relative to base_dir). REQUIRED.
    'pre_ranking_file': 'shuffled_pre_ranking.json', # Path to the initial ranking JSON (relative to base_dir).
    'queries_content_file': 'queries_content_with_features.json', # Path to queries content JSON (relative to base_dir).
    'documents_content_file': 'documents_content_with_features.json', # Path to documents content JSON (relative to base_dir).
    'output_file': 'prediction2.json', # Path to save the re-ranked prediction JSON (relative to base_dir).

    # --- Model and Text Settings ---
    'model_name': 'AI-Growth-Lab/PatentSBERTa', # Sentence Transformer model name.
    'pooling': 'mean', # Pooling strategy (Note: may be overridden by model config). Choices: 'mean', 'max', 'cls'
    'text_type': 'TA', # Type of text content. Choices: 'TA', 'claims', 'tac1', 'description', 'full', 'features'
    'max_length': 512, # Max sequence length for the model.

    # --- Execution Settings ---
    'batch_size': 32, # Batch size for encoding document texts.
    'device': None # Device: 'cuda', 'cpu', or None (auto-detect).
}

# --- Auto-detect device if not specified ---
if config['device'] is None:
    config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
elif config['device'] == 'cuda' and not torch.cuda.is_available():
    print("Warning: CUDA requested but not available. Using CPU.")
    config['device'] = 'cpu'

# ----------------------------
# Utility Functions
# ----------------------------

def load_json_file(file_path):
    """Load JSON data from a file"""
    print(f"Loading JSON from: {file_path}")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"Successfully loaded {len(data)} items.")
        return data
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {file_path}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred loading {file_path}: {e}")
        return None

def save_json_file(data, file_path):
    """Save data to a JSON file"""
    print(f"Saving JSON to: {file_path}")
    try:
        # Ensure the directory exists before saving
        output_dir = os.path.dirname(file_path)
        if output_dir: # Check if dirname returned a non-empty string
             os.makedirs(output_dir, exist_ok=True)
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2)
        print(f"Successfully saved data to {file_path}")
    except Exception as e:
        print(f"An error occurred saving to {file_path}: {e}")

def load_content_data(file_path):
    """Load content data from a JSON file and create a FAN to Content mapping."""
    data = load_json_file(file_path)
    if data is None:
        return {}

    content_dict = {}
    key_options = ['FAN', 'Application_Number'] # Handle potential key variations

    for item in data:
        fan_key = None
        for key in key_options:
            if key in item:
                # Sometimes Application_Number needs Application_Category appended
                if key == 'Application_Number' and 'Application_Category' in item:
                   fan_key = item[key] + item.get('Application_Category', '') # Safely get category
                else:
                   fan_key = item[key]
                break # Found a key, stop looking

        if fan_key and 'Content' in item:
             content_dict[fan_key] = item['Content']
        # else:
        #     print(f"Warning: Could not find FAN key or Content in item: {item.keys()}")

    print(f"Created content dictionary with {len(content_dict)} entries.")
    return content_dict


def extract_text(content_dict, text_type="TA"):
    """Extract text from patent content based on text_type"""
    if not isinstance(content_dict, dict):
        # print(f"Warning: Invalid content_dict provided (type: {type(content_dict)}), expected dict.")
        return ""

    text_parts = []

    # Note: The original argparse choices included 'TAC', but the function uses 'tac1'.
    # Adjust config['text_type'] if 'tac1' was intended instead of 'TAC'.
    if text_type in ["TA", "tac1", "full", "title_abstract"]:
        text_parts.append(content_dict.get("title", ""))
        text_parts.append(content_dict.get("pa01", "")) # Abstract

    if text_type in ["claims", "tac1", "full"]:
        claims = []
        first_claim = None
        # Sort keys to approximate claim order, although keys aren't guaranteed sequential
        sorted_keys = sorted([key for key in content_dict if key.startswith('c-')])
        for key in sorted_keys:
            claim_text = content_dict.get(key, "")
            if claim_text:
                claims.append(claim_text)
                if first_claim is None and text_type == "tac1":
                    first_claim = claim_text

        if text_type == "claims" or text_type == "full":
            text_parts.extend(claims)
        elif text_type == "tac1" and first_claim:
            text_parts.append(first_claim)

    if text_type in ["description", "full"]:
        # Add description paragraphs (keys starting with 'p')
        desc_parts = []
        # Sort keys to approximate paragraph order
        sorted_keys = sorted([key for key in content_dict if key.startswith('p')])
        for key in sorted_keys:
             desc_parts.append(content_dict.get(key,""))
        text_parts.extend(desc_parts)

    if text_type == "features":
        # Extract LLM features if present
        text_parts.append(content_dict.get("features", ""))

    # Join non-empty parts with a space
    return " ".join(filter(None, text_parts)).strip()


# ----------------------------
# Main Re-ranking Logic
# ----------------------------

# Changed function signature to accept config dictionary
def main(cfg):
    # --- Device Setup ---
    # Use device from the config dictionary
    device = torch.device(cfg['device'])
    print(f"Using device: {device}")

    # --- Construct Full Paths ---
    # Use base_dir from the config dictionary
    def get_full_path(path):
        if os.path.isabs(path):
            return path
        # Use cfg['base_dir'] instead of args.base_dir
        return os.path.join(cfg['base_dir'], path)

    # Use paths from the config dictionary
    query_list_file = get_full_path(cfg['query_list_file'])
    pre_ranking_file = get_full_path(cfg['pre_ranking_file'])
    queries_content_file = get_full_path(cfg['queries_content_file'])
    documents_content_file = get_full_path(cfg['documents_content_file'])
    output_file = get_full_path(cfg['output_file'])

    # --- Load Data ---
    query_ids = load_json_file(query_list_file)
    pre_ranking_data = load_json_file(pre_ranking_file)
    queries_content = load_content_data(queries_content_file)
    documents_content = load_content_data(documents_content_file)

    if not query_ids or not pre_ranking_data or not queries_content or not documents_content:
        print("Error: Failed to load one or more essential data files. Exiting.")
        return

    # --- Load Model ---
    # Use model_name from the config dictionary
    print(f"Loading SentenceTransformer model: {cfg['model_name']}")
    try:
        # Define model architecture if needed (e.g., for specific pooling)
        # word_embedding_model = models.Transformer(cfg['model_name'], max_seq_length=cfg['max_length'])
        # pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode=cfg['pooling']) # Use cfg['pooling']
        # model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device=device)

        # Simpler loading if default pooling (mean) is okay or model config handles it
        model = SentenceTransformer(cfg['model_name'], device=device)
        model.max_seq_length = cfg['max_length'] # Set max length using cfg['max_length']
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model {cfg['model_name']}: {e}")
        return

    # --- Re-ranking Process ---
    # Use text_type from the config dictionary
    print(f"Starting re-ranking for {len(query_ids)} queries using '{cfg['text_type']}' content...")
    results = {}
    missing_query_content = 0
    missing_pre_ranking = 0
    queries_with_no_valid_docs = 0

    for query_id in tqdm(query_ids, desc="Processing queries"):
        # 1. Get Query Content
        query_content_dict = queries_content.get(query_id)
        if not query_content_dict:
            # print(f"Warning: Content not found for query {query_id}")
            missing_query_content += 1
            results[query_id] = [] # Assign empty list if query content missing
            continue

        # Use text_type from config
        query_text = extract_text(query_content_dict, cfg['text_type'])
        if not query_text:
            # print(f"Warning: Extracted text is empty for query {query_id} with type '{cfg['text_type']}'")
            missing_query_content += 1
            results[query_id] = []
            continue

        # 2. Get Candidate Documents
        candidate_doc_ids = pre_ranking_data.get(query_id)
        if not candidate_doc_ids:
            # print(f"Warning: Pre-ranking not found for query {query_id}")
            missing_pre_ranking += 1
            results[query_id] = []
            continue

        # 3. Get Candidate Document Content
        doc_texts = []
        valid_doc_ids_for_query = []
        missing_docs_count = 0
        for doc_id in candidate_doc_ids:
            doc_content_dict = documents_content.get(doc_id)
            if not doc_content_dict:
                # print(f"Warning: Content not found for document {doc_id} (query {query_id})")
                missing_docs_count += 1
                continue

            # Use text_type from config
            doc_text = extract_text(doc_content_dict, cfg['text_type'])
            if doc_text:
                doc_texts.append(doc_text)
                valid_doc_ids_for_query.append(doc_id)
            else:
                 # print(f"Warning: Extracted text is empty for document {doc_id} with type '{cfg['text_type']}' (query {query_id})")
                 missing_docs_count += 1


        if not valid_doc_ids_for_query:
            # print(f"Warning: No valid document texts found for query {query_id} after checking {len(candidate_doc_ids)} candidates.")
            queries_with_no_valid_docs += 1
            results[query_id] = [] # Assign empty list if no valid docs
            continue

        # 4. Generate Embeddings (On-the-fly)
        try:
            # Use batch_size from config
            query_embedding = model.encode(
                query_text,
                convert_to_tensor=True,
                show_progress_bar=False,
                batch_size=1 # Batch size for query is usually 1
            )
            doc_embeddings = model.encode(
                doc_texts,
                convert_to_tensor=True,
                show_progress_bar=False,
                batch_size=cfg['batch_size'] # Use cfg['batch_size']
            )
        except Exception as e:
            print(f"Error during encoding for query {query_id}: {e}")
            results[query_id] = candidate_doc_ids # Fallback to original order on error
            continue


        # 5. Calculate Similarities
        # Ensure embeddings are on the same device for cosine similarity
        query_embedding = query_embedding.to(device)
        doc_embeddings = doc_embeddings.to(device)

        cosine_scores = util.cos_sim(query_embedding, doc_embeddings)[0] # Get the first row of scores
        cosine_scores = cosine_scores.cpu().numpy() # Move scores to CPU for sorting

        # 6. Rank Documents
        # Combine scores with their original valid doc_ids
        doc_scores = list(zip(valid_doc_ids_for_query, cosine_scores))

        # Sort by score in descending order
        doc_scores.sort(key=lambda x: x[1], reverse=True)

        # Get the sorted list of document IDs
        re_ranked_doc_ids = [doc_id for doc_id, score in doc_scores]

        # If some original docs were missing content, append their IDs at the end
        # (or handle differently if needed - e.g., exclude them)
        original_candidate_set = set(candidate_doc_ids)
        reranked_set = set(re_ranked_doc_ids)
        missing_from_reranked = list(original_candidate_set - reranked_set)
        final_ranked_list = re_ranked_doc_ids + missing_from_reranked

        results[query_id] = final_ranked_list[:len(candidate_doc_ids)] # Ensure max length is original candidate count


    # --- Report Missing Data ---
    print("\n--- Re-ranking Summary ---")
    print(f"Total queries processed: {len(query_ids)}")
    if missing_query_content > 0:
        print(f"Warning: Content missing or empty for {missing_query_content} queries.")
    if missing_pre_ranking > 0:
        print(f"Warning: Pre-ranking data missing for {missing_pre_ranking} queries.")
    if queries_with_no_valid_docs > 0:
        print(f"Warning: {queries_with_no_valid_docs} queries had no valid documents with content.")
    print(f"Number of queries in results: {len(results)}")


    # --- Save Results ---
    # Use output_file from config
    save_json_file(results, output_file)

    print("\nRe-ranking complete.")

main(config)

Using device: cuda
Loading JSON from: ./test_queries.json
Successfully loaded 10 items.
Loading JSON from: ./shuffled_pre_ranking.json
Successfully loaded 30 items.
Loading JSON from: ./queries_content_with_features.json
Successfully loaded 30 items.
Created content dictionary with 30 entries.
Loading JSON from: ./documents_content_with_features.json
Successfully loaded 900 items.
Created content dictionary with 900 entries.
Loading SentenceTransformer model: AI-Growth-Lab/PatentSBERTa
Model loaded successfully.
Starting re-ranking for 10 queries using 'TA' content...


Processing queries: 100%|██████████| 10/10 [00:05<00:00,  1.80it/s]


--- Re-ranking Summary ---
Total queries processed: 10
Number of queries in results: 10
Saving JSON to: ./prediction2.json
Successfully saved data to ./prediction2.json

Re-ranking complete.
